## franchise.ftc.go.kr 에 있는 프랜차이즈 가맹사업거래 정보공개서 내용을 가져오는 파이썬 코드 for vkim of 꿈데디

#####작성에 앞서 한가지. 본 코드를 작성한 필자는 파이썬을 배운지 얼마 되지 않았으며 codestudy와 주변의 도움으로 독학했다. 따라서 본 코드는 성능이나 유지의 편리함 보다는 '실행되는지?'에 초점이 맞춰져 있으며, 이렇게 실행되는 코드를 짜는데도 꽤 많은 시간이 걸렸음을 눈물로 호소하고 싶다. 며칠 밤샜다.  
#####본 notebook은 코드를 모두 작성한 뒤에 작성되었다. 따라서 코드에 대한 설명만 진행하므로 좌절의 순간이 담긴 '작업기'는 아님을 미리 밝혀둔다.

#####2015.5.29 작성자 박재현(Jstring90@gmail.com)

###그럼 시작!

필요한 패키지를 import한다. 코드는 파이썬2.7 기준으로 작성되었다.

urllib의 quote, unquote와 sys 패키지는 모두 인코딩 때문에 사용했다. 코드 작성중에 인코딩으로 매우 열받는 일이 많이 발생했는데, 문제를 모두 말끔히 해결해주는 착한 패키지이다.

In [1]:

# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup as BS
from urllib2 import urlopen  
from urllib import quote, unquote
import csv
import sys
reload(sys)
sys.setdefaultencoding('utf-8')


<br>
정보를 알고싶은 매장명을 카테고리별로 리스트를 만들어 입력한다. 뒤에 나올 파이썬 코드는 해당 리스트에서 매장명을 받아서 검색하게 되고 검색 첫 번째 링크에 들어있는 정보공개서 데이터를 긁어오는 역할을 한다.




In [2]:
franchiseList_Chicken = ['BBQ', '페리카나', '네네치킨', '교촌치킨', '처갓집양념치킨', '굽네치킨', '또래오래', 'BHC', '호식이두마리', '부어치킨', '맥시칸치킨', '맘스터치', '지코바']
franchiseList_Coffee = ['카페띠아모','파스쿠찌','할리스','탐앤탐스','투썸플레이스','엔제리너스','이디야','카페베네']
franchiseList_Ddokk = ['아딸','죠스떡볶이','올떡','동대문엽기떡볶이','국대떡볶이','버무리떡볶이','요런떡볶이','신전떡볶이','신참떡볶이']

사용할 카테고리는 딕셔너리 형태로 저장. 후에 key값과 value값 모두 사용해야한다.

In [3]:
category = {'Chicken':franchiseList_Chicken,'Ddokk':franchiseList_Ddokk,'Coffee':franchiseList_Coffee}


<br>이제 페이지 크롤링하는 코드를 작성한다. 우선 프랜차이즈 카테고리 별로 각각의 csv를 저장하도록 반복문을 작성한다. 
csv 패키지를 사용하며 파일 저장 이름은 카테고리 이름으로 지정했다. 
csv 패키지 함수에는 close가 없으므로 새롭게 open을 하기 전까지 나오는 cw.writerow 함수의 내용을 모두 자신의 데이터로 인식한다. 
<br>

html_List라는 빈 리스트는 페이지의 테그를 모두 저장해두는 역할을 한다.



In [4]:
for cat in range(len(category)):

    html_List = []
    csv_file = open(str(category.keys()[cat])+'.csv', "w")
    cw = csv.writer(csv_file, delimiter=',')

###여기까지는 크롤링을 위한 준비 단계라고 할 수 있다. 이제 본격적으로 크롤링 코드를 작성한다. 

<br>
우선 url과 관련한 코드를 살펴보자.

In [5]:
  for fran in category.values()[cat]:
        insert_name = quote(fran.encode('EUC-KR'))
        Url1 = 'http://franchise.ftc.go.kr/fir/manage/searchFirList2.do?method=getSearchList&currpage=1&indus=&t_nm=&brd=' 
        Url2 = '&stdate=&enddate=&onelimit=10'
        baseUrl = Url1 + insert_name+ Url2
        html = urlopen(baseUrl)
        bs = BS(html)

        Url3 = 'http://franchise.ftc.go.kr'
        number = bs.select('td.lef a')[1].attrs['href']
        final = Url3 + number
        bs2 = urlopen(final)
        bs3  = BS(bs2)

        html_List.append(bs3)


<br>먼저, 데이터를 긁어오려는 페이지의 주소정보를 입력해주고 필요한 정보가 들어있는 부분을 select를 써서 가져온다. url과 관련된 코드는 크게 윗부분 아랫부분으로 나누어진다. 먼저 윗부분을 보자.<br> 

In [6]:
        ae = quote(fran.encode('EUC-KR'))
        Url1 = 'http://franchise.ftc.go.kr/fir/manage/searchFirList2.do?method=getSearchList&currpage=1&indus=&t_nm=&brd=' 
        Url2 = '&stdate=&enddate=&onelimit=10'
        baseUrl = Url1 + ae + Url2
        html = urlopen(baseUrl)
        bs = BS(html)


윗 부분은 우리가 원하는 브랜드의 이름을 검색하여 해당 검색의 결과값을 보여주는 페이지를 읽고 그 페이지에서 해당 브랜드의 정보공개서 주소를 가져오게 된다. 


<br>
본 코드에서 필자를 가장 열받게 한 부분이 바로 insert_name부분이다. 우선 가맹사업거래 홈페이지에서 검색어가 들어가는 주소를 GET방식으로 받아내고 검색어가 들어갈 부분을 찾아서 insert_name으로 설정해줬다. 하지만 인코딩의 문제로 정상적으로 주소가 입력되지 않는 문제가 계속 발생했다. 
<br>문자열을 'EUC-KR'로 받아오고 urllib패키지의 quote모듈을 통해 해결하였다. quote모듈은 문자를 URL에 사용되는 형식으로 인코딩 하는 역할을 한다. 

In [7]:
      insert_name= quote(fran.encode('EUC-KR'))

<br>
다음은 아래부분이다. 위의 코드에서 bs로 해당 브랜드의 검색 결과를 받아왔다. 이제 검색 결과에서 해당 브랜드의 페이지로 연결되는 링크를 뽑아낼 차례다. 어렵지 않게 작성하고 bs3에 우리가 진짜 필요로 했던 해당 브랜드의 정보공개서 페이지 주소를 저장했다. 변수명을 너무 대충지었다. 다음부턴 신경써서 하겠다.

마지막 줄에 html_List에 데이터를 append하는 코드가 있다. 눈여겨 두면 나중에 다시 등장한다.

In [8]:
        Url3 = 'http://franchise.ftc.go.kr'
        number = bs.select('td.lef a')[1].attrs['href']
        final = Url3 + number
        bs2 = urlopen(final)
        bs3  = BS(bs2)
        
        html_List.append(bs3)

이제 정보공개 페이지로 들어왔으니 해당 브랜드의 데이터를 뽑아내는 코드를 작성해야한다. 공정위 정보공개 페이지는 테이블로 구성되어 있다. 우리가 필요로하는 테이블의 번호를 알아내서 해당 테이블에서 데이터를 뽑아내도록 하자. 테이블 번호는 다음과 같다. 페이지에 테이블이 왜이렇게 많은지 아무튼 테이블 번호 알아내는것도 은근 힘든 작업이었다. 
(참고로 테이블 번호를 알아낼 때 구글시트에서 importHTML 코드를 사용하는것이 가장 편했다.)

- 매장수: 32<br>
- 변동사항: 34<br>
- 지역: 36<br>
- 가맹비: 46<br>

총 네개의 데이터인데 그 중 매장수 데이터를 뽑아내는 코드만 살펴보자.

In [ ]:
cw.writerow(['매장수','년도','가맹점','직영점','계','년도','가맹점','직영점','계','년도','가맹점','직영점','계'])
    for List in category.values()[cat]:
        ls=[List.encode('utf-8'),]
        for i in html_List[category.values()[cat].index(List)].select('table')[32].select('td')[4 :16]:
            ls.append(i.text.encode('utf-8'))
        cw.writerow(ls)

우선 상단에 cw.writerow 모듈을 사용하면 csv파일에 하나의 행(row)를 쓰겠다는 말이다. 첫 행은 물론 해당 데이터의 칼럼명이 나와야한다. 이후 반복문을 통해서 데이터를 가져오게 되는데 설명하기가 솔직히 좀 귀찮다. 노가다의 연속이었다. 어려운 부분은 여러 매장의 데이터를 어떻게 한줄씩 읽어올까 하는 부분이었는데, <br><br>


In [ ]:
html_List[category.values()[cat].index(List)]

<br>바로 위의 부분이 고민을 해결하게 한 코드이다. 처음에 html_List에 해당 카테고리 모든 매장의 html테그를 받아 둔 후에 뒤에 csv로 작성하는 부분에서 html_List에 들어있는 모든 매장의 데이터를 한번 에 활용하는 아이디어를 냈다. 물론 처음에 모든 매장을 검색해서 html을 불러오기 때문에 초반 실행속도는 느리지만 뒷 부분에서 한번에 전체 매장의 데이터를 쓸 수 있는 장점이 있다. 
<br><br>
이렇게 네번 각각 테이블에서 비슷한 코드를 돌려주면 모든 데이터가 csv파일로 들어가게 된다. 



In [ ]:
cw.writerow(['매장수','년도','가맹점','직영점','계','년도','가맹점','직영점','계','년도','가맹점','직영점','계'])
    for List in category.values()[cat]:
        ls=[List.encode('utf-8'),]
        for i in html_List[category.values()[cat].index(List)].select('table')[32].select('td')[4 :16]:
            ls.append(i.text.encode('utf-8'))
        cw.writerow(ls)
    cw.writerow(['가맹점 변동사항','년도','신규','계약종료','계약해지','명의이전','년도','신규','계약종료','계약해지','명의이전','년도','신규','계약종료','계약해지','명의이전'])
    for List in category.values()[cat]:
        ls=[List.encode('utf-8'),]
        for i in html_List[category.values()[cat].index(List)].select('table')[34].select('td')[5 :25]:
            ls.append(i.text.encode('utf-8'))
        cw.writerow(ls)
    cw.writerow(['지역별','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액'])
    for List in category.values()[cat]:
        ls=[List.encode('utf-8'),]
        for i in html_List[category.values()[cat].index(List)].select('table')[36].select('td')[3 :57]:
            ls.append(i.text.encode('utf-8'))
        cw.writerow(ls)
    cw.writerow(['부담금','가입비(가맹비)','교육비','보증금','기타비용','계']) 
    for List in category.values()[cat]:
        ls=[List.encode('utf-8'),]
        for i in html_List[category.values()[cat].index(List)].select('table')[46].select('td')[5 :15]:
            ls.append(i.text.encode('utf-8'))
        cw.writerow(ls)


<br><br>설명이 길었다. 하지만 코드는 짧다. 전체 코드의 모습은 다음과 같다. 

##최종 코드 

In [6]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup as BS
from urllib2 import urlopen  
from urllib import quote, unquote
import csv
import sys
reload(sys)
sys.setdefaultencoding('utf-8')


franchiseList_Chicken = ['BBQ', '페리카나', '네네치킨', '교촌치킨', '처갓집양념치킨', '굽네치킨', '또래오래', 'BHC', '호식이두마리', '부어치킨', '맥시칸치킨', '맘스터치', '지코바']
franchiseList_Coffee = ['카페띠아모','파스쿠찌','할리스','탐앤탐스','투썸플레이스','엔제리너스','이디야','카페베네']
franchiseList_Ddokk = ['아딸','죠스떡볶이','올떡','동대문엽기떡볶이','국대떡볶이','버무리떡볶이','요런떡볶이','신전떡볶이','신참떡볶이']

category = {'Chicken':franchiseList_Chicken,'Ddokk':franchiseList_Ddokk,'Coffee':franchiseList_Coffee}

                  

for cat in range(len(category)):

    html_List = []
    csv_file = open(str(category.keys()[cat])+'.csv', "w")
    cw = csv.writer(csv_file, delimiter=',')


    for fran in category.values()[cat]:
        insert_name = quote(fran.encode('EUC-KR'))
        Url1 = 'http://franchise.ftc.go.kr/fir/manage/searchFirList2.do?method=getSearchList&currpage=1&indus=&t_nm=&brd=' 
        Url2 = '&stdate=&enddate=&onelimit=10'
        baseUrl = Url1 + insert_name + Url2
        html = urlopen(baseUrl)
        bs = BS(html)

        Url3 = 'http://franchise.ftc.go.kr'
        number = bs.select('td.lef a')[1].attrs['href']
        final = Url3 + number
        bs2 = urlopen(final)
        bs3  = BS(bs2)

        html_List.append(bs3)

        
    
    cw.writerow(['매장수','년도','가맹점','직영점','계','년도','가맹점','직영점','계','년도','가맹점','직영점','계'])
    for List in category.values()[cat]:
        ls=[List.encode('utf-8'),]
        for i in html_List[category.values()[cat].index(List)].select('table')[32].select('td')[4 :16]:
            ls.append(i.text.encode('utf-8'))
        cw.writerow(ls)
    cw.writerow(['가맹점 변동사항','년도','신규','계약종료','계약해지','명의이전','년도','신규','계약종료','계약해지','명의이전','년도','신규','계약종료','계약해지','명의이전'])
    for List in category.values()[cat]:
        ls=[List.encode('utf-8'),]
        for i in html_List[category.values()[cat].index(List)].select('table')[34].select('td')[5 :25]:
            ls.append(i.text.encode('utf-8'))
        cw.writerow(ls)
    cw.writerow(['지역별','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액','지역','가맹점수','평균매출액'])
    for List in category.values()[cat]:
        ls=[List.encode('utf-8'),]
        for i in html_List[category.values()[cat].index(List)].select('table')[36].select('td')[3 :57]:
            ls.append(i.text.encode('utf-8'))
        cw.writerow(ls)
    cw.writerow(['부담금','가입비(가맹비)','교육비','보증금','기타비용','계']) 
    for List in category.values()[cat]:
        ls=[List.encode('utf-8'),]
        for i in html_List[category.values()[cat].index(List)].select('table')[46].select('td')[5 :15]:
            ls.append(i.text.encode('utf-8'))
        cw.writerow(ls)


<br>이것으로 공정거래위원회 가맹거래사업 정보공개서 크롤링 코드에 대한 설명을 마친다. 
파이썬에 대한 공부를 좀 더 해서 더 좋은 코드를 만들면 다음에 꼭 다시 소개하도록 하겠다.
<br><br>
#####어쨌거나, 지금은 이게 최선이다. 

<br><br><br>